## CallBacks
Every statement in Javascript will execute "now" or "later". Callbacks are commonly used to specify code that will execute later. To illustrate the "now" and "later" parts,

In [ ]:
// A. "now" 

setTimeout(function(){
    // B. "later"
}, 0)

// C. "now"

The common complaint with callbacks is the notion of *callback hell*, callbacks nested inside callbacks. However callbacks are bad not due to multiple nesting it can create as we can see below:

In [ ]:
button.addEventListener('click', function(event){
    setTimeout(function(){
        ajax('http://someresource.com', function(response){
            if(response.status == 200){
                // Success
            } else if(response.status == 400){
                // Wrong request
            } else if(response.status == 500){
                // Server error
            }
        })
    }, 500)
})

In [ ]:
button.addEventListener('click', clickListener)

function clickListener(event){
    setTimeout(sendRequest, 500)
}

function sendRequest(){
    ajax('http://someresource.com', responseHandler)
}

function responseHandler(response){
    if(response.status == 200){
        // Success
    } else if(response.status == 400){
        // Wrong request
    } else if(response.status == 500){
        // Server error
    }
}

The real issue with callbacks is "trust issue" and not deep indentation. The problem with callback is inversion of control. The execution of callback function is controlled by part not directly under our control. For example in above code snippet, each of the functions `addEventListener`, `setTimeout` and `ajax` decide when to call the callback. This inversion of control means the following scenarios may develop (not necessarily for the above mentioned functions, but for some 3rd party function):
- callback called too early
- callback called too late
- callback not called
- callback called multiple times
- errors not handled

There are variants of callback which lets us handle errors.

In [ ]:
// Split error and success handling
ajax(url, success, failure)

function success(response){
    // implementation
}

function failure(err){
    // implementation
}

// Node style
ajax(url, function(err, response){
    if(err){
        // handle error
    } else {
        // work with response data
    }
})

## Promise
Represents a future value. Its constructor in action:

In [ ]:
const promise = new Promise(function(resolve, reject) {
    // calls either resolve with a value resolve("Some value")
    // or throw an error throw new Error("Failed");
    // or instead of throwing, use reject("Failed")
});

A promise can be in one of the following states:
- pending
- resolved
- rejected
 
A promise is settled if it is resolved or rejected. Example below shows a promise that is immediately resolved:

In [1]:
new Promise((resolve, reject) => resolve("Hello World!"));

// Static function that acts as shortcut:
Promise.resolve("Hello World!");

Promise { 'Hello World!' }


Another example of a promise that resolves in 10 seconds:

In [2]:
new Promise((resolve, reject) => {
    setTimeout(() => resolve(10), 10000)
});

Promise { <pending> }


What happens if we directly return a value instead of using `resolve`? The value is ignored.

In [7]:
const p = new Promise((resolve, reject) => {
  return 42; // ignored, this promise will never settle
});

What if resolve is called twice?

In [9]:
new Promise((resolve, reject) => {resolve("first"); resolve("seocond")}).then(console.log)

Promise { <pending> }
first


**`then`:** we can define an action that happens once the promise is settled using `then` method:

In [3]:
Promise.resolve({msg: "done"})
    .then(x => console.log(x));

Promise { <pending> }
{ msg: 'done' }


`then` can also handle promise in rejected state:

In [ ]:
// Prints Failed msg: failed
Promise.reject({msg: "failed"})
    .then(
        val => console.log(`Success msg: ${val.msg}`),
        err => console.log(`Failed msg: ${err.msg}`)
    );

`then` returns another promise. The return value of `then` depends on what happens inside its callback.

In [ ]:
// Callback returns a value
Promise.resolve(5)
    .then(x => x * 2) // returns 10
    .then(console.log); // logs 10

In [ ]:
// Callback throws
Promise.resolve(5)
    .then(() => { throw new Error("Oops"); })
    .catch(err => console.log(err.message)); // "Oops"

In [ ]:
// Callback returns a Promise
Promise.resolve(5)
    .then(x => Promise.resolve(x * 2))
    .then(console.log); // logs 10

In [ ]:
// Callback doesn't return
Promise.resolve(5)
    .then(() => {}) // returns undefined
    .then(x => console.log(x)); // undefined

In [ ]:
// Callback not defined
Promise.resolve(5)
    .then()  // pass through
    .then(console.log) // logs 5

**`catch`:** is syntax sugar for `.then(undefined, onRejected)`.

In [10]:
Promise.reject("error")
    .catch(console.error)

Promise { <pending> }


error


Like `then`, `catch` also returns a promise:

In [ ]:
// Callback returns a value
Promise.reject("error")
    .catch(err => {
        console.log("Caught:", err);
        return 50;   // Creates a promise that immediately resolved with 50
    })
    .then(console.log);  // logs 50

In [ ]:
// Callback throws error
Promise.reject("error")
  .catch(() => { throw "still error"; })
  .catch(err => console.log(err)); // logs "still error"

In [ ]:
// Callback returns another Promise
Promise.reject("oops")
    .catch(() => Promise.resolve("recovered"))
    .then(console.log); // "recovered"

In [ ]:
// Callback doesn't return
Promise.reject("oops")
    .catch(console.log) // "oops"
    .then(console.log); // "undefined"

In [ ]:
// Callback not defined
Promise.reject("error")
    .catch()
    .then(
        console.log,
        console.error  // logs "error"
    );

**`finally`:** executes everytime regardless of whether the promise resolved or rejected. Used to perform cleanup.

In [12]:
Promise.resolve("Data")
    .then(value => {
        console.log(`In then, received value ${value}`);
        throw new Error("Data Error");
    })
    .catch(err => {
        console.error(`In catch, error is ${err}`);
    })
    .finally(() => {  // Does not accept any argument
        console.log(`In finally`);
    });

Promise { <pending> }
In then, received value Data


In catch, error is Error: Data Error


In finally


In [ ]:
// Original value passes through
Promise.resolve(5)
    .finally(() => console.log("Cleanup"))
    .then(x => console.log("Value is ", x));  // 5

In [ ]:
// Value returned by finally is ignored
Promise.resolve(5)
    .finally(() => 10)  // Or Promise.resolve(10)
    .then(x => console.log("Value is ", x)); // 5

In [ ]:
// Finally throws an error
Promise.resolve(5)
    .finally(() => { throw "DataError"; })    // Or Promise.reject("DataError")
    .then(x => console.log("Value is ", x))   // Not printed
    .catch(e => console.log("Error is ", e)); // Prints Error is DataError

### Promise API
**`Promise.all`:** accepts an iterable of promises and returns a promise which resolves when all the promises passed got resolved.

In [14]:
let requests = Promise.all([
    fetch("https://www.google.com"),
    fetch("https://www.wikipedia.org"),
    fetch("https://www.github.com")
]);

requests.then(responses => {
    responses.forEach(response => {
        console.log(`${response.url} returned status ${response.status}`);
    });
});

Promise { <pending> }
https://www.google.com/ returned status 200
https://www.wikipedia.org/ returned status 200
https://github.com/ returned status 200


When any one of the passed promise is rejected, the final promise is also rejected.

In [15]:
requests = Promise.all([
    fetch("https://www.google.com"),  // result ignored
    fetch("https://bananas"),
    fetch("https://www.github.com")   // result ignored
]);

requests.then(responses => {
        responses.forEach(response => {
            console.log(`${response.url} returned status ${response.status}`);
        });
    })
    .catch(err => {
        console.error(`Failed to load ${err.cause.hostname}, reason ${err.cause.code}`);
    });

Promise { <pending> }


Failed to load bananas, reason ENOTFOUND


In the above example, loading abcd failed, but it did not cancel loading of the other two resources (which is not something we would have liked).

You can also pass non-promise values:

In [17]:
Promise.all([1, 2, 3]).then(values => values.forEach(x => console.log(x)));

Promise { <pending> }
1
2
3


**`Promise.allSettled`:** unlike `Promise.all`, `Promise.allSettled` does not reject if one of the promise gets rejected. It waits for all promises to get settled and depending upon whether the promise got resolved or rejected, we get the following entries in the resulting array:
- `{status:"fulfilled", value:result}` for successful responses,
- `{status:"rejected", reason:error}` for errors.

In [ ]:
requests = Promise.allSettled([
    fetch("https://www.google.com"),
    fetch("https://bananas"),
    fetch("https://www.github.com")
]);

requests.then(responses => {
        responses.forEach(response => {
            if(response.status === "fulfilled")
                console.log(`${response.value.url} returned status ${response.value.status}`);
            else if(response.status === "rejected")
                console.error(`Failed to load ${response.reason.cause.hostname}, reason ${response.reason.cause.code}`);
        });
    })

/* Prints
https://www.google.com/ returned status 200
Failed to load bananas, reason ENOTFOUND
https://github.com/ returned status 200
*/

**`Promise.race`:** get the fastest resolving (or rejected) promise. Rest are ignored.

In [21]:
let anotherRequests = Promise.race([
    fetch("https://www.wikipedia.org"),
    fetch("https://www.google.com"),
    fetch("https://www.github.com")
]);

anotherRequests.then(response => console.log(`Response from ${response.url} received fastest`));

Promise { <pending> }
Response from https://github.com/ received fastest


In [22]:
anotherRequests = Promise.race([
    fetch("https://www.wikipedia.org"),
    Promise.reject("Some error"),
    fetch("https://www.github.com")
]);

anotherRequests.then(
    response => console.log(`Response from ${response.url} received fastest`),
    error => console.log(`Got error: ${error}`)
);

Promise { <pending> }
Got error: Some error


**`Promise.any`:** similar to `Promise.race` but waits only for the first *fulfilled* promise and gets its result. If all of the given promises are rejected, then the returned promise is rejected with `AggregateError`.

In [23]:
anotherRequests = Promise.any([
    fetch("https://www.wikipedia.org"),
    Promise.reject("Some error"),
    fetch("https://www.github.com")
]);

anotherRequests.then(
    response => console.log(`Response from ${response.url} received fastest`)
);

Promise { <pending> }
Response from https://github.com/ received fastest


In [27]:
// All promises failed
let failingRequests = Promise.any([
    Promise.reject("First error"),
    Promise.reject("Second error"),
    Promise.reject("Third error")
]);

failingRequests.then(
    response => console.log(response),
    error => error.errors.forEach(e => console.log(e))
);

Promise { <pending> }
First error
Second error
Third error


## Async Await
`async` keyword added to a function defintion results in converting everything returned by the function to a promise.

In [28]:
async function asyncSquare(number) {
    return number*number
}

asyncSquare(25)
    .then(console.log);

Promise { <pending> }
625


We can also return explicit promise:

In [29]:
async function asyncSquare2(number) {
    return Promise.resolve(number*number)
}

asyncSquare2(25)
    .then(console.log);

Promise { <pending> }
625


`await` keyword used only inside an `async` function causes the JavaScript runtime to pause your code on this line, allowing other code to execute in the meantime, until the async function call has returned its result.

In [ ]:
// Code using await
const value = await somePromise();
doSomething(value);

// Is equivalent to
somePromise().then(value => {
  doSomething(value);
});

Infact when When Javascript sees an `await` inside an `async` function, it effectively transforms it into promise chaining.

In [ ]:
async function demo() {
  const x = await foo();
  console.log(x);
}

// Is transformed to
function demo() {
  return foo().then(x => {
    console.log(x);
  });
}

// Example with exception
async function demoWithException() {
  try {
    const data = await fetchData();
    console.log(data);
  } catch (err) {
    console.error(err);
  }
}

// Is transformed to
function demoWithException() {
    fetchData()
      .then(data => console.log(data))
      .catch(err => console.error(err));
}

A more practical example:

In [ ]:
// Promise version
fetch('www.google.com')
.then(function(response){
    if(response.status == 200){
        return fetch('www.yahoo.com')
    } else if(response.status >= 400){
        throw Error('Error in request/server')
    }
})
.then(function(response){
    console.log('Obtained response successfully')
})
.catch(function(error){
    console.log('Error found ', error)
})

// Async await version
async function fetchSites(){
    let response = await fetch('www.google.com') // Execution pauses here
    
    if(response.status == 200){
        response = await fetch('www.yahoo.com')
        console.log('Obtained response successfully')
    } else if(response.status >= 400){
        throw Error('Error in request/server')
    }
}

fetchSites()
    .catch(function(error){  // Or wrap above function in try catch
        console.log('Error found ', error)
    })

Async await should be used with some care. If multiple awaits are used each one will wait for the previous one to complete. Consider the below code:

In [2]:
function timeoutPromise(delay, name){
    return new Promise(function(resolve, reject){
        setTimeout(function(){
            resolve(name + ' resolved')
        }, delay)
    })
}

// Slow version
async function slower(){
    let a = await timeoutPromise(200, 'A')
    console.log(a)
    let b = await timeoutPromise(400, 'B')
    console.log(b)
    let c = await timeoutPromise(600, 'C')
    console.log(c)
}

let start = Date.now()
slower()
.then(function(){
    let end = Date.now()
    console.log('Time taken: ', end - start)
})

Promise { <pending> }
A resolved
B resolved
C resolved
Time taken:  1218


In [3]:
// Faster version
async function faster(){
    let aP = timeoutPromise(200, 'A')
    let bP = timeoutPromise(400, 'B')
    let cP = timeoutPromise(600, 'C')
    
    let a = await aP
    console.log(a)
    let b = await bP
    console.log(b)
    let c = await cP
    console.log(c)
}

let start = Date.now()
faster()
.then(function(){
    let end = Date.now()
    console.log('Time taken: ', end - start)
})

Promise { <pending> }
A resolved
B resolved
C resolved
Time taken:  604


The first version is functionally equivalent to:

In [ ]:
let start = Date.now()
timeoutPromise(200, 'A')
  .then(x => {
    console.log(x);
    return timeoutPromise(400, 'B');
  })
  .then(y => {
    console.log(y);
    return timeoutPromise(600, 'C');
  })
  .then(z => {
    console.log(z);
  })
  .finally(() => {
      let end = Date.now();
      console.log(`Time taken: ${end - start}`);
  });

Whereas the second one is equivalent to:

In [ ]:
let start = Date.now()
timeoutPromise(200, 'A')
  .then(x => {
    console.log(x);
  });

timeoutPromise(400, 'B')
  .then(y => {
    console.log(y);
  });

timeoutPromise(600, 'C')
  .then(z => {
    console.log(z);
  })
  .finally(() => {
      let end = Date.now();
      console.log(`Time taken: ${end - start}`);
  });